# AudioEngine - Professional Therapeutic Noise Generator

This notebook demonstrates how to use the AudioEngine to generate studio-quality therapeutic noise content optimized for YouTube.

## Features:
- Continuous audio generation (no segments or loops)
- ITU-R BS.1770-4 compliant loudness (-14 LUFS for YouTube)
- Infant-optimized frequency shaping
- GPU acceleration for fast processing
- Professional 48kHz/24-bit FLAC output

## 1. Installation and Setup

First, install the required dependencies:

In [ ]:
# For Google Colab
!pip install torch torchaudio librosa soundfile numpy scipy psutil

# Optional: For full metadata support
!pip install mutagen

In [ ]:
# Import the audio engine modules
import sys
import os

# Add the AudioEngine directory to path if needed
# sys.path.append('/path/to/AudioEngine')

from audio_engine import NoiseGenerator
import torch
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Audio, display

## 2. Check GPU Availability

In [ ]:
# Check if CUDA is available
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 3. Initialize the Noise Generator

In [ ]:
# Initialize with default settings (optimized for YouTube)
generator = NoiseGenerator(
    sample_rate=48000,      # YouTube native sample rate
    bit_depth=24,           # Professional quality
    target_lufs=-14.0,      # YouTube reference level
    use_cuda=True,          # Enable GPU acceleration
    therapeutic_eq=True,    # Apply infant-optimized EQ
    fade_duration=5.0,      # 5-second fade in/out
    oversampling_factor=4   # 4x oversampling for quality
)

# Display device information
device_info = generator.get_device_info()
print("Device Configuration:")
for key, value in device_info.items():
    print(f"  {key}: {value}")

## 4. Generate Different Types of Noise

### 4.1 White Noise

In [ ]:
# Generate 5 minutes of white noise
print("Generating white noise...")
white_noise = generator.generate_white_noise(duration_minutes=5)

# Export to FLAC
generator.export_flac(
    filename="white_noise_5min.flac",
    audio=white_noise,
    noise_type="white",
    duration_minutes=5
)

# Preview (first 5 seconds)
preview_samples = 5 * generator.config.sample_rate
display(Audio(white_noise[:, :preview_samples].cpu().numpy(), rate=generator.config.sample_rate))

### 4.2 Pink Noise

In [ ]:
# Generate 10 minutes of pink noise
print("Generating pink noise...")
pink_noise = generator.generate_pink_noise(duration_minutes=10)

# Export to FLAC
generator.export_flac(
    filename="pink_noise_10min.flac",
    audio=pink_noise,
    noise_type="pink",
    duration_minutes=10
)

# Preview
display(Audio(pink_noise[:, :preview_samples].cpu().numpy(), rate=generator.config.sample_rate))

### 4.3 Brown Noise

In [ ]:
# Generate 30 minutes of brown noise
print("Generating brown noise...")
brown_noise = generator.generate_brown_noise(duration_minutes=30)

# Export to FLAC
generator.export_flac(
    filename="brown_noise_30min.flac",
    audio=brown_noise,
    noise_type="brown",
    duration_minutes=30
)

# Preview
display(Audio(brown_noise[:, :preview_samples].cpu().numpy(), rate=generator.config.sample_rate))

## 5. Visualize Spectral Characteristics

In [ ]:
def plot_spectrum(audio, title, sample_rate=48000):
    """Plot the frequency spectrum of audio"""
    # Take a segment for analysis
    segment = audio[0, :sample_rate*2].cpu().numpy()  # 2 seconds
    
    # Compute FFT
    fft = np.fft.rfft(segment)
    freqs = np.fft.rfftfreq(len(segment), 1/sample_rate)
    magnitude = 20 * np.log10(np.abs(fft) + 1e-10)
    
    # Plot
    plt.figure(figsize=(10, 6))
    plt.semilogx(freqs[1:], magnitude[1:])  # Skip DC
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude (dB)')
    plt.title(f'Frequency Spectrum - {title}')
    plt.grid(True, alpha=0.3)
    plt.xlim(20, 20000)
    plt.show()

# Plot spectra for each noise type
plot_spectrum(white_noise, "White Noise")
plot_spectrum(pink_noise, "Pink Noise")
plot_spectrum(brown_noise, "Brown Noise")

## 6. Generate Long-Duration Content for YouTube

In [ ]:
# Generate 1-hour content for YouTube
print("Generating 1-hour pink noise for YouTube...")
print("This will create a continuous, uninterrupted audio stream.")

# Generate
youtube_audio = generator.generate_pink_noise(duration_minutes=60)

# Export with metadata
generator.export_flac(
    filename="pink_noise_1hour_youtube.flac",
    audio=youtube_audio,
    noise_type="pink",
    duration_minutes=60
)

print("\nFile ready for YouTube upload!")
print("- Format: 48kHz/24-bit FLAC")
print("- Loudness: -14 LUFS (YouTube standard)")
print("- True Peak: ≤-1 dBTP")
print("- Duration: 60 minutes")
print("- Generation: Continuous (no loops or segments)")

## 7. Custom Configuration Examples

In [ ]:
# Example 1: Generate without therapeutic EQ
generator_flat = NoiseGenerator(
    therapeutic_eq=False,  # No infant-optimized EQ
    use_cuda=True
)

flat_white = generator_flat.generate_white_noise(duration_minutes=5)
print("Generated flat white noise (no therapeutic EQ)")

In [ ]:
# Example 2: Generate for different loudness target
generator_quiet = NoiseGenerator(
    target_lufs=-20.0,  # Quieter output
    use_cuda=True
)

quiet_pink = generator_quiet.generate_pink_noise(duration_minutes=5)
print("Generated quieter pink noise (-20 LUFS)")

## 8. Batch Processing Example

In [ ]:
# Generate a collection of different durations
durations = [30, 60, 120, 180]  # minutes
noise_types = ['white', 'pink', 'brown']

print("Generating noise collection...")
for noise_type in noise_types:
    for duration in durations:
        print(f"\nGenerating {duration} minutes of {noise_type} noise...")
        
        # Generate based on type
        if noise_type == 'white':
            audio = generator.generate_white_noise(duration_minutes=duration)
        elif noise_type == 'pink':
            audio = generator.generate_pink_noise(duration_minutes=duration)
        else:
            audio = generator.generate_brown_noise(duration_minutes=duration)
        
        # Export
        filename = f"{noise_type}_noise_{duration}min.flac"
        generator.export_flac(
            filename=filename,
            audio=audio,
            noise_type=noise_type,
            duration_minutes=duration
        )
        
        print(f"Exported: {filename}")
        
        # Clear GPU cache between generations
        if generator.cuda.enabled:
            generator.cuda.clear_cache()

print("\nCollection complete!")

## 9. Memory Management for Ultra-Long Content

In [ ]:
# For very long content (e.g., 10+ hours), process in chunks
def generate_ultra_long_content(hours, noise_type='pink'):
    """Generate ultra-long content by processing in memory-efficient chunks"""
    
    total_minutes = hours * 60
    chunk_minutes = 60  # Process 1 hour at a time
    
    print(f"Generating {hours} hours of {noise_type} noise...")
    print(f"Processing in {chunk_minutes}-minute chunks for memory efficiency")
    
    # This is a demonstration - in practice, you'd want to stream
    # the output directly to disk rather than concatenating in memory
    
    for chunk_idx in range(0, total_minutes, chunk_minutes):
        current_chunk_minutes = min(chunk_minutes, total_minutes - chunk_idx)
        print(f"\nProcessing chunk {chunk_idx//chunk_minutes + 1}/{total_minutes//chunk_minutes}")
        
        # Generate chunk
        if noise_type == 'white':
            chunk = generator.generate_white_noise(duration_minutes=current_chunk_minutes)
        elif noise_type == 'pink':
            chunk = generator.generate_pink_noise(duration_minutes=current_chunk_minutes)
        else:
            chunk = generator.generate_brown_noise(duration_minutes=current_chunk_minutes)
        
        # Export chunk
        chunk_filename = f"{noise_type}_noise_{hours}h_chunk{chunk_idx//chunk_minutes + 1}.flac"
        generator.export_flac(
            filename=chunk_filename,
            audio=chunk,
            noise_type=noise_type,
            duration_minutes=current_chunk_minutes
        )
        
        # Clear memory
        del chunk
        if generator.cuda.enabled:
            generator.cuda.clear_cache()
    
    print(f"\nCompleted {hours}-hour generation!")

# Example: Generate 3 hours
# generate_ultra_long_content(3, 'brown')

## 10. Cleanup

In [ ]:
# Clear GPU cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("GPU cache cleared")